In [1]:
# !pip install torch
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.metrics import confusion_matrix
import torch
import torch.nn as nn
import torch.optim as optim

/home/panw/miniconda3/envs/gsMap/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
df = pd.read_csv('./Shoppers_Behaviour_and_Revenue.csv')
df.head()

,Administrative,Administrative_Duration,Informational,Informational_Duration,ProductRelated,ProductRelated_Duration,BounceRates,ExitRates,PageValues,SpecialDay,Month,OperatingSystems,Browser,Region,TrafficType,VisitorType,Weekend,Revenue
0,0,0.0,0,0.0,1,0.000000,0.20,0.20,0.0,0.0,Feb,1,1,1,1,Returning_Visitor,False,False
1,0,0.0,0,0.0,2,64.000000,0.00,0.10,0.0,0.0,Feb,2,2,1,2,Returning_Visitor,False,False
2,0,0.0,0,0.0,1,0.000000,0.20,0.20,0.0,0.0,Feb,4,1,9,3,Returning_Visitor,False,False
3,0,0.0,0,0.0,2,2.666667,0.05,0.14,0.0,0.0,Feb,3,2,2,4,Returning_Visitor,False,False
4,0,0.0,0,0.0,10,627.500000,0.02,0.05,0.0,0.0,Feb,3,3,1,4,Returning_Visitor,True,False


In [3]:
# Encode categorical features
label_cols = ['Month', 'VisitorType', 'Weekend', 'Revenue']
le = LabelEncoder()

for col in label_cols:
    df[col] = le.fit_transform(df[col])

# Split features and target
X = df.drop("Revenue", axis=1)
y = df["Revenue"]

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [4]:
X_train = torch.tensor(X_train.values, dtype = torch.float32)  # longTensor, FloatTensor, bf16, fp16
X_test = torch.tensor(X_test.values, dtype = torch.float32)
y_train = torch.tensor(y_train.values, dtype = torch.float32)
# y_test = torch.tensor(y_test.values.tolist(), dtype = torch.float32)

In [5]:
print("X_train.shape", X_train.shape)
print("y_train.shape", y_train.shape)

X_train.shape torch.Size([9864, 17])
y_train.shape torch.Size([9864])


In [355]:
# 归一化
# root mean square normalization
# layer normalization 
# pytorch 实现

In [6]:
y_train = y_train.unsqueeze(1)
# y_test = y_test.unsqueeze(1)
y_train.shape

torch.Size([9864, 1])

In [8]:
 # linear regression =  y = w^T * X + b
class DeepNeuralNetwork(nn.Module):   # 一定要继承nn.Module, python class构件，python构件一个类的instance：__new__(), __init__(), 这两个方法调用的先后顺序是怎么样的？
    def __init__(self, input_size, hidden1_size, hidden2_size):
        # 在 DeepNeuralNetwork.__init__ 里，去调用它的父类 nn.Module 的 __init__, python不会自动调用父类中的构造函数;
        # https://developer.aliyun.com/article/1467527
        super(DeepNeuralNetwork, self).__init__()
        # 第一层：输入 -> 隐藏层1
        self.layer1 = nn.Linear(input_size, hidden1_size)   # pytorch  y = w^T * X + b（W \in R^{16 * 17}）
        # 第二层：隐藏层1 -> 隐藏层2
        self.layer2 = nn.Linear(hidden1_size, hidden2_size)  # W = R^{32 ^ 16}
        # 第三层：隐藏层2 -> 输出（1个神经元，用于二分类）
        self.output = nn.Linear(hidden2_size, 1)  # W = R^(1 * 32)
        # 激活函数  
        self.relu = nn.ReLU()
        # self.sigmoid = nn.Sigmoid()  # 将输出压缩到 [0,1]

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)  # Xavier 初始化适合 relu
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
                    
    def forward(self, x):
        x = self.relu(self.layer1(x))
        x = self.relu(self.layer2(x))
        x = self.output(x)
        # x = self.sigmoid(self.output(x))  # 输出概率
        return x

input_size = 17      
hidden1_size = 32
hidden2_size = 16
learning_rate = 0.005
num_epochs = 200  
batch_size = 4096   # 这里是所有的数据 mini_batch vs. full-batch


model = DeepNeuralNetwork(input_size, hidden1_size, hidden2_size)

# 使用 Binary Cross Entropy Loss
pos_weight = torch.tensor([5.0])  # 假设正样本少，权重高 https://blog.csdn.net/qq_22210253/article/details/85222093
criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
# criterion = nn.BCELoss(weight=pos_weight)

# 怎么样解决数据集imbalance 1. upsamping 2. downsampling 3. weighted loss

# 使用 Adam 优化器  1. 目前代码使用adam优化器 2. SGD优化器，momentum
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
# gradients，momentum

model.train()  # 切换到训练模式
for epoch in range(num_epochs):
    running_loss = 0.0
  
    outputs = model(X_train)
    loss = criterion(outputs, y_train)
    # 反向传播 + 优化
    optimizer.zero_grad()
    loss.backward() #调用返向传播算法
    optimizer.step()
    running_loss = loss.item()

    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss:.4f}')

print("✅ 训练完成！")
# hyparameters search: grid search

Epoch [10/200], Loss: 2.1633
Epoch [20/200], Loss: 1.4292
Epoch [30/200], Loss: 1.4060
Epoch [40/200], Loss: 0.9115
Epoch [50/200], Loss: 0.7644
Epoch [60/200], Loss: 0.7522
Epoch [70/200], Loss: 0.7455
Epoch [80/200], Loss: 0.7324
Epoch [90/200], Loss: 0.7901
Epoch [100/200], Loss: 0.7316
Epoch [110/200], Loss: 0.7161
Epoch [120/200], Loss: 0.7793
Epoch [130/200], Loss: 0.7181
Epoch [140/200], Loss: 0.7036
Epoch [150/200], Loss: 0.7728
Epoch [160/200], Loss: 0.7076
Epoch [170/200], Loss: 0.6875
Epoch [180/200], Loss: 0.7485
Epoch [190/200], Loss: 0.6938
Epoch [200/200], Loss: 0.6727
✅ 训练完成！


In [9]:
outputs_test = model(X_test)
outputs_test = outputs_test.squeeze(1)
outputs_test = outputs_test.tolist()
max(outputs_test)

25.466957092285156

In [10]:
# y_pred = [1 if i >= 0.5 else 0 for i in outputs_test]
y_pred = [1 if i >= 0 else 0 for i in outputs_test]

In [11]:
acc = accuracy_score(y_test, y_pred)
print("the accuracy: ", acc)
print("confusion matrix: \n", confusion_matrix(y_test, y_pred))
print("classification report: \n", classification_report(y_test, y_pred))

the accuracy:  0.878345498783455
confusion matrix: 
 [[1921  134]
 [ 166  245]]
classification report: 
               precision    recall  f1-score   support

           0       0.92      0.93      0.93      2055
           1       0.65      0.60      0.62       411

    accuracy                           0.88      2466
   macro avg       0.78      0.77      0.77      2466
weighted avg       0.87      0.88      0.88      2466

